##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# <a>Algoritmos federados personalizados, parte 1: introducción a Federated Core</a>

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_1"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/custom_federated_algorithms_1.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

Este tutorial es la primera parte de una serie de dos partes que demuestra cómo implementar tipos personalizados de algoritmos federados en TensorFlow Federated (TFF) con la ayuda de [Federated Core (FC)](../federated_core.md), un conjunto de interfaces de nivel inferior que sirven como base para implementar la capa de [Aprendizaje Federado (FL)](../federated_learning.md).

Esta primera parte es más conceptual; presentamos algunos de los conceptos clave y abstracciones de programación que se usan en TFF y demostramos su uso en un ejemplo muy simple con un arreglo distribuido de sensores de temperatura. En [la segunda parte de esta serie](custom_federated_algorithms_2.ipynb), usamos los mecanismos aquí presentados para implementar una versión simple de algoritmos federados de entrenamiento y evaluación. A modo de seguimiento, le recomendamos que estudie [la implementación](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/python/learning/algorithms/fed_avg.py) del promediado federado en `tff.learning`.

Al final de esta serie, debería ser capaz de reconocer que las aplicaciones de Federated Core no se limitan necesariamente al aprendizaje. Las abstracciones de programación que ofrecemos son bastante genéricas y podrían usarse, por ejemplo, para implementar análisis y otros tipos personalizados de cálculos sobre datos distribuidos.

Aunque este tutorial está diseñado para ser autónomo, le recomendamos que primero lea los tutoriales sobre [clasificación de imágenes](federated_learning_for_image_classification.ipynb) y [generación de texto](federated_learning_for_text_generation.ipynb) para acceder a una introducción más sencilla y general del marco federado de TensorFlow y las API de [Aprendizaje Federado](../federated_learning.md) (`tff.learning`), ya que esto le servirá para poner en contexto los conceptos que describimos aquí.

## Usos previstos

En pocas palabras, Federated Core (FC) es un entorno de desarrollo que permite expresar de forma compacta la lógica del programa que combina el código de TensorFlow con operadores de comunicación distribuida, como los que se utilizan en el [promediado federado](https://arxiv.org/abs/1602.05629): cálculo de sumas distribuidas, promedios y otros tipos de agregaciones distribuidas sobre un conjunto de dispositivos cliente en el sistema, difusión de modelos y parámetros a esos dispositivos, etc.

Es posible que conozca [`tf.contrib.distribute`](https://www.tensorflow.org/api_docs/python/tf/contrib/distribute) y entonces una pregunta natural sería: ¿en qué se diferencia este marco? Después de todo, ambos marcos intentan hacer que los cálculos de TensorFlow sean distribuidos.

Una forma de pensarlo es que, mientras que el objetivo expreso de `tf.contrib.distribute` es el de *permitir que los usuarios usen los modelos y el código de entrenamiento existentes con cambios mínimos para facilitar el entrenamiento distribuido*, y gran parte de la atención se centra en cómo aprovechar la infraestructura distribuida para hacer que el código de entrenamiento existente sea más eficiente, el objetivo de Federated Core de TFF es brindar a los investigadores y profesionales un control explícito sobre los patrones específicos de comunicación distribuida que utilizarán en sus sistemas. FC pone el énfasis en ofrecer un lenguaje flexible y extensible para expresar algoritmos de flujo de datos distribuidos, en lugar de un conjunto concreto de capacidades de entrenamiento distribuido implementadas.

Uno de los principales destinatarios de la API FC de TFF son los investigadores y profesionales que quieran experimentar con nuevos algoritmos de aprendizaje federado y evaluar las consecuencias de las sutiles elecciones de diseño que afectan la forma en que se organiza el flujo de datos en el sistema distribuido, pero sin perderse en los detalles de implementación del sistema. El nivel de abstracción al que aspira la API FC se corresponde aproximadamente con el pseudocódigo que uno podría usar para describir la mecánica de un algoritmo de aprendizaje federado en una publicación de investigación: qué datos existen en el sistema y cómo se transforman, pero sin caer al nivel de intercambios individuales de mensajes de red punto a punto.

TFF en su conjunto apunta a escenarios en los que los datos se distribuyen y deben conservar esa distribución, por ejemplo, por razones de privacidad, y en los que recopilar todos los datos en una ubicación centralizada puede no ser una opción viable. Esto repercute en la implementación de algoritmos de aprendizaje automático que requieren un mayor grado de control explícito, en comparación con escenarios en los que todos los datos se pueden acumular en una ubicación centralizada en un centro de datos.

## Antes de empezar

Antes de profundizar en el código, intente ejecutar el siguiente ejemplo "Hola mundo" para asegurarse de que su entorno esté configurado correctamente. Si no funciona, consulte la guía de [instalación](../install.md) para acceder a las instrucciones.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## Datos federados

Una de las características distintivas de TFF es que nos permite expresar de forma compacta cálculos basados ​​en TensorFlow sobre *datos federados*. En este tutorial, usaremos el término *datos federados* para referirnos a una colección de elementos de datos alojados en un grupo de dispositivos en un sistema distribuido. Por ejemplo, las aplicaciones que se ejecutan en dispositivos móviles pueden recopilar datos y almacenarlos localmente, sin cargarlos en una ubicación centralizada. O bien, un arreglo de sensores distribuidos puede recopilar y almacenar lecturas de temperatura en sus ubicaciones.

Los datos federados como los de los ejemplos anteriores se tratan en TFF como [ciudadanos de primera clase](https://en.wikipedia.org/wiki/First-class_citizen), es decir, pueden aparecer como parámetros y resultados de funciones, y tienen tipos. Para reforzar esta noción, nos referiremos a los conjuntos de datos federados como *valores federados* o *valores de tipos federados*.

Los más importante es comprender que estamos modelando toda la colección de elementos de datos en todos los dispositivos (por ejemplo, todas las lecturas de temperatura de la colección de todos los sensores en un arreglo distribuido) como un único valor federado.

Por ejemplo, así es como se definiría en TFF el tipo de *flotante federado* alojado por un grupo de dispositivos cliente. Una colección de lecturas de temperatura que se materializan en un arreglo de sensores distribuidos podría modelarse como un valor de este tipo federado.

In [ ]:
federated_float_on_clients = tff.type_at_clients(tf.float32)

En términos más generales, un tipo federado en TFF se define especificando el tipo `T` de sus *miembros constituyentes*: los elementos de datos que residen en dispositivos individuales y el grupo `G` de dispositivos en los que se alojan los valores federados de este tipo (además de una tercera información opcional que mencionaremos en breve). Nos referimos al grupo `G` de dispositivos que albergan un valor federado como la *ubicación* del valor. Por lo tanto, `tff.CLIENTS` es un ejemplo de ubicación.

In [ ]:
str(federated_float_on_clients.member)

'float32'

In [ ]:
str(federated_float_on_clients.placement)

'CLIENTS'

Un tipo federado con miembros constituyentes `T` y ubicación `G` se puede representar de forma compacta como `{T}@G`, tal y como se muestra a continuación.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

Las llaves `{}` en esta notación concisa sirven como recordatorio de que los miembros constituyentes (elementos de datos en diferentes dispositivos) pueden diferir, como es de esperar, por ejemplo, en las lecturas de los sensores de temperatura, por lo que los clientes como grupo organizan conjuntamente un [conjunto múltiple](https://en.wikipedia.org/wiki/Multiset) de elementos tipo `T` que en conjunto constituyen el valor federado.

Cabe destacar que los miembros constituyentes de un valor federado generalmente son opacos para el programador, es decir, un valor federado no debe considerarse como un simple `dict` introducido por un identificador de un dispositivo en el sistema; estos valores están destinados a ser transformados colectivamente solo por *operadores federados* que representen de manera abstracta varios tipos de protocolos de comunicación distribuida (como la agregación). Si esto suena demasiado abstracto, no se preocupe: lo retomaremos en breve para ilustrarlo con ejemplos concretos.

Hay dos tipos de tipos federados en TFF: aquellos en los que los miembros constituyentes de un valor federado pueden diferir (como se vio arriba) y aquellos en los que se sabe que todos son iguales. Esto está controlado por el tercer parámetro opcional `all_equal` en el constructor `tff.FederatedType` (el valor predeterminado es `False`).

In [ ]:
federated_float_on_clients.all_equal

False

Un tipo federado con una ubicación `G` en la que se sabe que todos los miembros constituyentes de tipo `T` son iguales se puede representar de forma compacta como `T@G` (a diferencia de `{T}@G`, es decir, se eliminan las llaves para indicar que el conjunto múltiple de miembros constituyentes está formado por un único elemento).

In [ ]:
str(tff.type_at_clients(tf.float32, all_equal=True))

'float32@CLIENTS'

Un ejemplo de un valor federado de este tipo que podría surgir en escenarios prácticos es un hiperparámetro (como una tasa de aprendizaje, una norma de recorte, etc.) que ha sido difundido por un servidor a un grupo de dispositivos que participan en el entrenamiento federado.

Otro ejemplo es un conjunto de parámetros para un modelo de aprendizaje automático previamente entrenado en el servidor, que luego se difunden a un grupo de dispositivos cliente, donde se pueden personalizar para cada usuario.

Por ejemplo, supongamos que tenemos un par de parámetros `float32` `a` y `b` para un modelo de regresión lineal unidimensional simple. Podemos construir el tipo (no federado) de dichos modelos para usar en TFF de la siguiente manera. Las llaves angulares `<>` en la cadena de tipos impresa son una notación compacta de TFF para tuplas nombradas y sin nombrar.

In [ ]:
simple_regression_model_type = (
    tff.StructType([('a', tf.float32), ('b', tf.float32)]))

str(simple_regression_model_type)

'<a=float32,b=float32>'

Tenga en cuenta que hasta ahora solo especificamos los `dtype`. También se admiten tipos no escalares. En el código anterior, `tf.float32` es una notación abreviada para un código `tff.TensorType(dtype=tf.float32, shape=[])` más general.

Cuando este modelo se difunde a los clientes, el tipo de valor federado resultante se puede representar como se muestra a continuación.

In [ ]:
str(tff.type_at_clients(
    simple_regression_model_type, all_equal=True))

'<a=float32,b=float32>@CLIENTS'

Por cuestiones de simetría con el *flotante federado* que mencionamos más arriba, nos referiremos a este tipo como *tupla federada*. En términos más generales, a menudo usaremos el término *XYZ federado* para referirnos a un valor federado en el que los miembros constituyentes son similares a *XYZ*. Por lo tanto, hablaremos de *tuplas federadas*, *secuencias federadas*, *modelos federados*, etc.

Ahora bien, volviendo a `float32@CLIENTS`: si bien parece que se replica en varios dispositivos, en realidad se trata de un único `float32`, ya que todos los miembros son iguales. En general, se puede pensar en cualquier tipo federado *totalmente igual*, es decir, uno de la forma `T@G`, como isomorfo a un tipo no federado `T`, ya que, en ambos casos, en realidad hay un solo elemento (aunque potencialmente replicado) de tipo `T`.

A raíz del isomorfismo entre `T` y `T@G`, cabe preguntarse para qué sirven estos últimos tipos, si es que sirven para algo. Continúe leyendo.

## Ubicaciones

### Descripción general del diseño

En la sección anterior, presentamos el concepto de *ubicaciones*: grupos de participantes del sistema que podrían albergar conjuntamente un valor federado, y demostramos el uso de `tff.CLIENTS` como ejemplo de especificación de una ubicación.

Para explicar por qué la noción de *ubicación* es tan fundamental que necesitábamos incorporarla al sistema de tipo TFF, recuerde lo que mencionamos al principio de este tutorial sobre algunos de los usos previstos de TFF.

Aunque en este tutorial solo verá una ejecución local del código TFF en un entorno simulado, nuestro objetivo es que TFF permita escribir código que se pueda implementar para su ejecución en grupos de dispositivos físicos en un sistema distribuido, lo que podría incluir dispositivos móviles o integrados con Android. Cada uno de esos dispositivos recibiría un conjunto independiente de instrucciones para ejecutar localmente, en función del papel que desempeñe en el sistema (un dispositivo de usuario final, un coordinador centralizado, una capa intermedia en una arquitectura de múltiples niveles, etc.). Resulta importante poder determinar qué subconjuntos de dispositivos ejecutan qué código y dónde pueden materializarse físicamente las distintas partes de los datos.

Esto es especialmente importante cuando se trata, por ejemplo, de datos de aplicaciones en dispositivos móviles. Dado que los datos son privados y pueden ser confidenciales, necesitamos la capacidad de verificar estáticamente que estos datos nunca saldrán del dispositivo (y comprobar la forma en que se procesan los datos). Las especificaciones de ubicación son uno de los mecanismos diseñados para conseguirlo.

TFF se diseñó como un entorno de programación centrado en datos y, como tal, a diferencia de algunos de los marcos existentes que se centran en las *operaciones* y dónde podrían *ejecutarse* esas operaciones, TFF se centra en los *datos*, dónde *se materializan* esos datos y cómo se *transforman*. En consecuencia, la ubicación se modela como una propiedad de los datos en TFF, y no como una propiedad de las operaciones con los datos. De hecho, como verá en la siguiente sección, algunas de las operaciones de TFF se extienden a través de ubicaciones y se ejecutan "en la red", por así decirlo, en lugar de que las ejecute una sola máquina o un grupo de máquinas.

Representar el tipo de un determinado valor como `T@G` o `{T}@G` (en lugar de hacerlo solo como `T`) hace que las decisiones de ubicación de datos sean explícitas y, junto con un análisis estático de los programas escritos en TFF, puede servir como base para proporcionar garantías formales de privacidad para los datos confidenciales en el dispositivo.

Sin embargo, algo importante a tener en cuenta en este punto es que, si bien recomendamos a los usuarios de TFF que sean explícitos sobre los *grupos* de dispositivos participantes que alojan los datos (las ubicaciones), el programador nunca tratará con los datos en bruto ni con las identidades de los participantes *individuales*.

Dentro del cuerpo del código TFF, por su propio diseño, no hay forma de enumerar los dispositivos que constituyen el grupo representado por `tff.CLIENTS` ni de investigar la existencia de un dispositivo específico en el grupo. No existe ningún concepto de identidad de cliente o dispositivo en ninguna parte de la API de Federated Core, el conjunto subyacente de abstracciones arquitectónicas o la infraestructura de ejecución básica que proporcionamos para admitir simulaciones. Toda la lógica de cálculo que escriba se expresará como operaciones en todo el grupo de clientes.

Recuerde aquí lo que mencionamos anteriormente acerca de que los valores de los tipos federados son diferentes a `dict` de Python, en el sentido de que uno no puede simplemente enumerar sus miembros constituyentes. Piense en los valores que usa la lógica de su programa de TFF como si estuvieran asociados con ubicaciones (grupos) y no con participantes individuales.

Las ubicaciones también *están* diseñadas como ciudadanos de primera clase en TFF y pueden aparecer como parámetros y resultados de un tipo `placement` (que se representarán mediante `tff.PlacementType` en la API). Está previsto que en el futuro ofrezcamos diversos operadores para transformar o combinar ubicaciones, pero esto queda fuera del alcance de este tutorial. Por ahora, basta con pensar en `placement` como un tipo integrado primitivo opaco en TFF, similar a la forma en que `int` y `bool` son tipos integrados opacos en Python, donde `tff.CLIENTS` es un literal constante de este tipo, de forma similar a como `1` es un literal constante de tipo `int`.

### Especificación de ubicaciones

TFF ofrece dos literales de ubicación básicos, `tff.CLIENTS` y `tff.SERVER`, para facilitar la expresión de una amplia variedad de escenarios prácticos que se modelan de forma natural como arquitecturas cliente-servidor, con múltiples dispositivos *cliente* (teléfonos móviles, dispositivos integrados, bases de datos distribuidas, sensores, etc.) orquestados por un único coordinador *de servidor* centralizado. El diseño de TFF también admite ubicaciones personalizadas, grupos de clientes múltiples, arquitecturas distribuidas en varios niveles y otras arquitecturas más generales, pero su análisis queda fuera del alcance de este tutorial.

TFF no prescribe lo que realmente representan `tff.CLIENTS` o `tff.SERVER`.

Concretamente, `tff.SERVER` puede ser un único dispositivo físico (un miembro de un grupo singleton), pero también podría ser un grupo de réplicas en un clúster con tolerancia a errores que ejecuta la réplica de máquinas de estado; no asumimos ninguna arquitectura especial. Más bien, utilizamos el bit `all_equal` mencionado en la sección anterior para expresar el hecho de que generalmente estamos tratando con un solo elemento de datos en el servidor.

Del mismo modo, `tff.CLIENTS` en algunas aplicaciones puede representar a todos los clientes del sistema, lo que en el contexto del aprendizaje federado a veces denominamos *población*, pero, por ejemplo, en [implementaciones de producción de promediado federado](https://arxiv.org/abs/1602.05629), podría representar una *cohorte*, un subconjunto de los clientes seleccionados para participar en una ronda de entrenamiento específica. Las ubicaciones que se definen de forma abstracta reciben un significado concreto cuando un cálculo en el que aparecen se implementa para su ejecución (o simplemente se invoca como una función de Python en un entorno simulado, como se demuestra en este tutorial). En nuestras simulaciones locales, el grupo de clientes está determinado por los datos federados que se suministran como entrada.

## Cálculos federados

### Cómo declarar cálculos federados

TFF se diseñó como un entorno de programación funcional fuertemente tipado que admite desarrollo modular.

La unidad básica de composición en TFF es un *cálculo federado*: una sección de lógica que acepta valores federados como entrada y devuelve valores federados como salida. A continuación, se explica cómo se puede definir un cálculo que calcule el promedio de las temperaturas informadas por el arreglo de sensores de nuestro ejemplo anterior.

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):
  return tff.federated_mean(sensor_readings)

Al observar el código anterior, en este punto es posible que se pregunte: ¿no existen ya construcciones decoradoras para definir unidades componibles como [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) en TensorFlow? Y, de ser así, ¿por qué introducir otra más y en qué se diferencia?

La respuesta breve es que el código generado por el contenedor `tff.federated_computation` *no* es TensorFlow *ni* Python; es una especificación de un sistema distribuido en un lenguaje *pegamento* interno independiente de la plataforma. En este punto, esto sin duda sonará críptico, pero tenga en cuenta esta interpretación intuitiva de un cálculo federado como una especificación abstracta de un sistema distribuido. Lo explicaremos en un minuto.

En primer lugar, pensemos un poco en la definición. Los cálculos de TFF generalmente se modelan como funciones, con o sin parámetros, pero con firmas de tipo bien definidas. Puede imprimir la firma de tipo de un cálculo al consultar su propiedad `type_signature`, como se muestra a continuación.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

La firma de tipo nos dice que el cálculo acepta una colección de diferentes lecturas de sensores en los dispositivos cliente y devuelve un promedio único al servidor.

Antes de continuar, reflexionemos sobre esto por un minuto: la entrada y la salida de este cálculo están *en lugares diferentes* (en los `CLIENTS` o en el `SERVER`). Recuerde lo que dijimos en la sección anterior sobre ubicaciones acerca de cómo *las operaciones de TFF pueden extenderse a través de ubicaciones y ejecutarse en la red*, y lo que acabamos de decir sobre los cálculos federados como representación de especificaciones abstractas de sistemas distribuidos. Hace un instante definimos uno de esos cálculos: un sistema distribuido simple en el que los datos se consumen en los dispositivos cliente y los resultados agregados emergen en el servidor.

En muchos ejemplos prácticos, los cálculos que representan tareas de nivel superior tenderán a aceptar sus entradas y reportar sus salidas en el servidor; esto refleja la idea de que los cálculos se pueden desencadenar a partir de *consultas* que se originan y terminan en el servidor.

Sin embargo, la API FC no impone este supuesto, y muchos de los componentes básicos que se usan internamente (incluidos numerosos operadores `tff.federated_...` que puede encontrar en la API) tienen entradas y salidas con distintas ubicaciones, por lo que, en general, debe no se debe pensar en un cálculo federado como algo que *se ejecuta en el servidor* o que *es ejecutado por un servidor*. El servidor es solo un tipo de participante en un cálculo federado. Al pensar en la mecánica de estos cálculos, es mejor adoptar siempre la perspectiva de una red global, en lugar de la perspectiva de un único coordinador centralizado.

En general, las firmas de tipo funcional se representan de forma compacta como `(T -> U)` para los tipos `T` y `U` de entradas y salidas, respectivamente. El tipo de parámetro formal (como `sensor_readings` en este caso) se especifica como argumento para el decorador. No es necesario especificar el tipo de resultado: se determina automáticamente.

Aunque TFF ofrece formas limitadas de polimorfismo, se recomienda enfáticamente a los programadores que sean explícitos sobre los tipos de datos con los que trabajan, ya que eso facilita la comprensión, la depuración y la verificación formal de las propiedades de su código. En algunos casos, especificar tipos explícitamente es un requisito (por ejemplo, los cálculos polimórficos actualmente no son directamente ejecutables).

### Cómo ejecutar cálculos federados

Para respaldar el desarrollo y la depuración, TFF le permite invocar directamente los cálculos definidos de esta manera como funciones de Python, como se muestra a continuación. Cuando el cálculo espera un valor de un tipo federado con el bit `all_equal` establecido en `False`, puede cargarlo como una `list` simple en Python, y para los tipos federados con el bit `all_equal` establecido en `True`, puede cargar directamente el (único) miembro constituyente. Así es también como se le informan los resultados.

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

Cuando se ejecutan cálculos de este tipo en modo de simulación, se actúa como un observador externo con una vista de todo el sistema, que tiene la capacidad de suministrar entradas y usar salidas en cualquier ubicación de la red, como de hecho es el caso aquí: usted suministró valores del cliente en la entrada y usó el resultado del servidor.

Ahora bien, volvamos a una nota que hicimos anteriormente sobre el decorador `tff.federated_computation` que emite código en un lenguaje *pegamento*. Aunque la lógica de los cálculos de TFF se puede expresar como funciones comunes en Python (solo necesita decorarlas con `tff.federated_computation` como lo hicimos anteriormente), y puede invocarlas directamente con argumentos de Python como cualquier otra función de Python en este bloc de notas, en realidad, como señalamos anteriormente, los cálculos de TFF en realidad *no* son Python.

Lo que queremos decir con esto es que cuando el intérprete de Python encuentra una función decorada con `tff.federated_computation`, rastrea las instrucciones en el cuerpo de esta función una vez (en el momento de la definición) y luego construye una [representación serializada](https://github.com/tensorflow/federated/blob/main/tensorflow_federated/proto/v0/computation.proto) de la lógica de cálculo para usarla más adelante, ya sea para ejecutarla o para incorporarla como un subcomponente en otro cálculo.

Puede verificar esto si agrega una instrucción de impresión, de la siguiente manera:

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def get_average_temperature(sensor_readings):

  print ('Getting traced, the argument is "{}".'.format(
      type(sensor_readings).__name__))

  return tff.federated_mean(sensor_readings)

Getting traced, the argument is "Value".


Puede pensar en el código Python que define un cálculo federado como si fuera código Python que construye un gráfico de TensorFlow en un contexto no eager (si no está familiarizado con los usos distintos de eager de TensorFlow, piense en su código Python como un código que define un gráfico de operaciones que se ejecutarán más tarde, pero no las ejecuta sobre la marcha). El código de creación de gráficos no eager en TensorFlow es Python, pero el gráfico de TensorFlow construido con este código es independiente de la plataforma y se puede serializar.

Del mismo modo, los cálculos de TFF se definen en Python, pero las instrucciones de Python en sus cuerpos, como `tff.federated_mean` en el ejemplo que acabamos de mostrar, se compilan en una representación serializable portátil e independiente de la plataforma subyacente.

Como desarrollador, no tiene que preocuparse por los detalles de esta representación, ya que nunca tendrá que trabajar directamente con ella, pero debe ser consciente de su existencia, del hecho de que los cálculos de TFF son fundamentalmente no eager y no pueden capturar el estado arbitrario de Python. El código Python contenido en el cuerpo de un cálculo TFF se ejecuta en el momento de la definición, cuando se rastrea el cuerpo de la función Python decorada con `tff.federated_computation` antes de serializarse. No se vuelve a rastrear en el momento de la invocación (excepto cuando la función es polimórfica; consulte las páginas de documentación para obtener más detalles).

Quizás se pregunte por qué elegimos introducir una representación interna dedicada que no sea Python. Una razón es que, en última instancia, los cálculos de TFF están pensados para implementarse en entornos físicos reales y alojarse en dispositivos móviles o integrados, donde Python puede no estar disponible.

Otra razón es que los cálculos de TFF expresan el comportamiento global de los sistemas distribuidos, a diferencia de los programas Python, que expresan el comportamiento local de los participantes individuales. Eso se puede ver en el sencillo ejemplo anterior, donde el operador especial `tff.federated_mean` acepta datos en dispositivos cliente, pero deposita los resultados en el servidor.

El operador `tff.federated_mean` no se puede modelar fácilmente como un operador común en Python, ya que no se ejecuta localmente; como se señaló anteriormente, representa un sistema distribuido que coordina el comportamiento de múltiples participantes del sistema. Nos referiremos a dichos operadores como *operadores federados*, para distinguirlos de los operadores comunes (locales) en Python.

El sistema de tipos de TFF, y el conjunto fundamental de operaciones admitidas en el lenguaje de TFF, se desvía significativamente del que se usa en Python, lo que requiere el uso de una representación específica.

### Cómo componer cálculos federados

Como se señaló anteriormente, los cálculos federados y sus componentes se entienden mejor como modelos de sistemas distribuidos, y se puede pensar en la composición de cálculos federados como la composición de sistemas distribuidos más complejos a partir de otros más simples. Puede pensar en el operador `tff.federated_mean` como una especie de plantilla integrada de cálculo federado con una firma de tipo `({T}@CLIENTS -> T@SERVER)` (de hecho, al igual que los cálculos que usted escribe, este operador también tiene una estructura compleja; internamente lo dividimos en operadores más simples).

Lo mismo ocurre con la composición de cálculos federados. El cálculo `get_average_temperature` se puede invocar en el cuerpo de otra función de Python decorada con `tff.federated_computation`; al hacerlo, se insertará en el cuerpo de la función primaria, de la misma manera que `tff.federated_mean` se insertó anteriormente en su propio cuerpo.

Una restricción importante que se debe tener en cuenta es que los cuerpos de las funciones de Python decoradas con `tff.federated_computation` deben contener *únicamente* operadores federados, es decir, no pueden contener operaciones de TensorFlow de forma directa. Por ejemplo, no pueden usar directamente las interfaces `tf.nest` para agregar un par de valores federados. El código de TensorFlow debe limitarse a bloques de código decorados con `tff.tf_computation`, que se analiza en la siguiente sección. Solo cuando se envuelve de esta manera se puede invocar el código de TensorFlow envuelto en el cuerpo de un `tff.federated_computation`.

Esta separación se debe tanto a cuestiones técnicas (es difícil engañar a operadores como `tf.add` para que trabajen con no tensores) como arquitectónicas. El lenguaje de cálculos federados (es decir, la lógica construida a partir de cuerpos serializados de funciones de Python decoradas con `tff.federated_computation`) se diseñó para funcionar como un lenguaje *pegamento* independiente de la plataforma. Este lenguaje pegamento se utiliza actualmente para construir sistemas distribuidos a partir de secciones integradas del código de TensorFlow (limitadas a bloques `tff.tf_computation`). Con el tiempo, anticipamos la necesidad de incorporar secciones de otra lógica que no sea de TensorFlow, como consultas de bases de datos relacionales que podrían representar canalizaciones de entrada, todas conectadas entre sí mediante el mismo lenguaje pegamento (los bloques `tff.federated_computation`).

## Lógica de TensorFlow

### Cómo declarar cálculos de TensorFlow

TFF fue diseñado para usarse con TensorFlow. Por lo tanto, es probable que la mayor parte del código que se escriba en TFF sea código de TensorFlow común (es decir, de ejecución local). Para usar este código con TFF, como se indicó anteriormente, solo debe decorarlo con `tff.tf_computation`.

Por ejemplo, así es como podríamos implementar una función que tome un número y le agregue `0.5`.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

Una vez más, al observar esto, es posible que se pregunte por qué tendríamos que definir otro decorador `tff.tf_computation` en lugar de usar directamente un mecanismo existente como `tf.function`. A diferencia de la sección anterior, aquí tratamos con un bloque normal de código de TensorFlow.

Hay varias razones para esto, que se escapan al alcance de este tutorial, pero vale la pena nombrar la principal:

- Con el fin de insertar bloques de construcción reutilizables implementados con código de TensorFlow en los cuerpos de los cálculos federados (por ejemplo, que se rastreen y serialicen en el momento de la definición, que tengan firmas de tipo, etc.). Para esto, generalmente se requiere algún tipo de decorador.

En general, recomendamos el uso de los mecanismos nativos de TensorFlow para la composición, como `tf.function`, siempre que sea posible, ya que cabe esperar que la forma exacta en la que el decorador de TFF interactúa con las funciones eager evolucione.

Ahora, volviendo al fragmento de código del ejemplo anterior, TFF puede tratar el cálculo `add_half` que acabamos de definir como cualquier otro cálculo de TFF. En particular, tiene una firma de tipo TFF.

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

Tenga en cuenta que este tipo de firma no tiene ubicaciones. Los cálculos de TensorFlow no pueden usar ni devolver tipos federados.

Ahora también puede usar `add_half` como bloque de construcción en otros cálculos. Por ejemplo, a continuación se muestra cómo se puede usar el operador `tff.federated_map` para aplicar `add_half` puntualmente a todos los miembros constituyentes de un flotante federado en dispositivos cliente.

In [ ]:
@tff.federated_computation(tff.type_at_clients(tf.float32))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

### Cómo ejecutar cálculos de TensorFlow

La ejecución de los cálculos definidos con `tff.tf_computation` sigue las mismas reglas que las que describimos para `tff.federated_computation`. Se pueden invocar como invocables comunes en Python, de la siguiente manera.

In [ ]:
add_half_on_clients([1.0, 3.0, 2.0])

[<tf.Tensor: shape=(), dtype=float32, numpy=1.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.5>]

Una vez más, cabe señalar que invocar el cálculo `add_half_on_clients` de esta manera simula un proceso distribuido. Los datos se consumen en los clientes y se devuelven a los clientes. De hecho, este cálculo hace que cada cliente ejecute una acción local. No hay ningún `tff.SERVER` que se mencione explícitamente en este sistema (incluso si en la práctica, orquestar dicho procesamiento podría involucrar uno). Piense en un cálculo definido de esta manera como conceptualmente análogo a la etapa `Map` en `MapReduce`.

Además, tenga en cuenta que lo que dijimos en la sección anterior sobre la serialización de los cálculos de TFF en el momento de la definición también es válido para el código `tff.tf_computation`: el cuerpo del cálculo `add_half_on_clients` de Python se rastrea una vez en el momento de la definición. En invocaciones posteriores, TFF utiliza su representación serializada.

La única diferencia entre los métodos de Python decorados con `tff.federated_computation` y los decorados con `tff.tf_computation` es que estos últimos se serializan como gráficos de TensorFlow (mientras que los primeros no admiten la inserción directa de código de TensorFlow).

A nivel interno, cada método decorado con `tff.tf_computation` desactiva temporalmente la ejecución eager para permitir que se capture la estructura del cálculo. Si bien la ejecución eager está deshabilitada localmente, puede utilizar construcciones eager de TensorFlow, AutoGraph, TensorFlow 2.0, etc., siempre y cuando la lógica del cálculo se escriba de tal manera que se pueda serializar correctamente.

Por ejemplo, el siguiente código generará un error:

In [ ]:
try:

  # Eager mode
  constant_10 = tf.constant(10.)

  @tff.tf_computation(tf.float32)
  def add_ten(x):
    return x + constant_10

except Exception as err:
  print (err)

Attempting to capture an EagerTensor without building a function.


El código anterior da error porque `constant_10` ya se construyó fuera del gráfico que `tff.tf_computation` construye internamente en el cuerpo de `add_ten` durante el proceso de serialización.

Por otro lado, es correcto invocar funciones de Python que modifican el gráfico actual cuando se llaman dentro de `tff.tf_computation`:

In [ ]:
def get_constant_10():
  return tf.constant(10.)

@tff.tf_computation(tf.float32)
def add_ten(x):
  return x + get_constant_10()

add_ten(5.0)

15.0

Tenga en cuenta que los mecanismos de serialización en TensorFlow están evolucionando y esperamos que los detalles relativos al modo en que TFF serializa los cálculos también evolucionen.

### Cómo trabajar con `tf.data.Dataset`

Como se señaló anteriormente, una característica única de los cálculos `tff.tf_computation` es que nos permiten trabajar con `tf.data.Dataset` definidos de manera abstracta como parámetros formales del código. Los parámetros que se representarán en TensorFlow como conjuntos de datos deben declararse mediante el constructor `tff.SequenceType`.

Por ejemplo, la especificación de tipo `tff.SequenceType(tf.float32)` define una secuencia abstracta de elementos flotantes en TFF. Las secuencias pueden contener tensores o estructuras anidadas complejas (más adelante explicaremos algunos ejemplos). La representación concisa de una secuencia de elementos escritos en `T` es `T*`.

In [ ]:
float32_sequence = tff.SequenceType(tf.float32)

str(float32_sequence)

'float32*'

Supongamos que, en nuestro ejemplo de sensor de temperatura, cada sensor contiene no solo una lectura de temperatura, sino varias. A continuación, se explica cómo se puede definir un cálculo de TFF en TensorFlow para que calcule el promedio de temperaturas en un único conjunto de datos local a partir del operador `tf.data.Dataset.reduce`.

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def get_local_temperature_average(local_temperatures):
  sum_and_count = (
      local_temperatures.reduce((0.0, 0), lambda x, y: (x[0] + y, x[1] + 1)))
  return sum_and_count[0] / tf.cast(sum_and_count[1], tf.float32)

In [ ]:
str(get_local_temperature_average.type_signature)

'(float32* -> float32)'

En el cuerpo de un método decorado con `tff.tf_computation`, los parámetros formales de un tipo de secuencia de TFF se representan simplemente como objetos que se comportan como `tf.data.Dataset`, es decir, admiten las mismas propiedades y métodos (actualmente no están implementados como subclases de ese tipo; esto puede cambiar a medida que evolucione la compatibilidad con conjuntos de datos en TensorFlow).

Esto se puede verificar fácilmente de la siguiente manera.

In [ ]:
@tff.tf_computation(tff.SequenceType(tf.int32))
def foo(x):
  return x.reduce(np.int32(0), lambda x, y: x + y)

foo([1, 2, 3])

6

Tenga en cuenta que, a diferencia de los `tf.data.Dataset` comunes, estos objetos similares a conjuntos de datos son marcadores de posición. No contienen ningún elemento, ya que representan parámetros abstractos de tipo secuencia, que se vincularán a datos específicos cuando se utilicen en un contexto concreto. La compatibilidad con los conjuntos de datos de tipo marcador de posición definidos de forma abstracta es todavía un tanto limitada en este momento, y en los primeros días de TFF, es posible que encuentre ciertas restricciones, pero no tendremos que preocuparnos por ellas en este tutorial (consulte las páginas de documentación para obtener más información).

Al ejecutar localmente un cálculo que acepta una secuencia en modo de simulación, como en este tutorial, puede cargar la secuencia como una lista de Python, como se muestra a continuación (así como de otras maneras, por ejemplo, como `tf.data.Dataset` en modo eager, pero por ahora, lo simplificaremos).

In [ ]:
get_local_temperature_average([68.5, 70.3, 69.8])

69.53333

Como todos los demás tipos de TFF, secuencias como las que definimos anteriormente pueden usar el constructor `tff.StructType` para definir estructuras anidadas. Por ejemplo, así es como se podría declarar un cálculo que acepte una secuencia de pares `A`, `B` y devuelva la suma de sus productos. Incluimos las instrucciones de seguimiento en el cuerpo del cálculo para que pueda ver cómo la firma de tipo TFF se traduce en `output_types` y `output_shapes` del conjunto de datos.

In [ ]:
@tff.tf_computation(tff.SequenceType(collections.OrderedDict([('A', tf.int32), ('B', tf.int32)])))
def foo(ds):
  print('element_structure = {}'.format(ds.element_spec))
  return ds.reduce(np.int32(0), lambda total, x: total + x['A'] * x['B'])

element_structure = OrderedDict([('A', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('B', TensorSpec(shape=(), dtype=tf.int32, name=None))])


In [ ]:
str(foo.type_signature)

'(<A=int32,B=int32>* -> int32)'

In [ ]:
foo([{'A': 2, 'B': 3}, {'A': 4, 'B': 5}])

26

La compatibilidad con el uso de `tf.data.Datasets` como parámetros formales todavía es algo limitada y está en desarrollo, aunque es funcional en escenarios simples como los que se usan en este tutorial.

## Unimos todo

Ahora, volvamos a tratar de usar el cálculo de TensorFlow en un entorno federado. Supongamos que tenemos un grupo de sensores y cada uno de ellos tiene una secuencia local de lecturas de temperatura. Podemos calcular el promedio de temperatura global al promediar las medias locales de los sensores de la siguiente manera.

In [ ]:
@tff.federated_computation(
    tff.type_at_clients(tff.SequenceType(tf.float32)))
def get_global_temperature_average(sensor_readings):
  return tff.federated_mean(
      tff.federated_map(get_local_temperature_average, sensor_readings))

Recuerde que no se trata de una simple media de todas las lecturas locales de temperatura de todos los clientes, ya que eso requeriría sopesar las contribuciones de diferentes clientes por la cantidad de lecturas que almacenan localmente. Le dejamos como ejercicio actualización del código anterior; el operador `tff.federated_mean` acepta la ponderación como segundo argumento opcional (se espera que sea un flotante federado).

También debe tener en cuenta que la entrada a `get_global_temperature_average` ahora se convierte en una *secuencia flotante federada*. Las secuencias federadas constituyen la forma en que normalmente representaremos los datos del dispositivo en el aprendizaje federado, con elementos de secuencia que normalmente representan lotes de datos (pronto veremos ejemplos de esto).

In [ ]:
str(get_global_temperature_average.type_signature)

'({float32*}@CLIENTS -> float32@SERVER)'

Así es como podemos ejecutar localmente el cálculo en una muestra de datos en Python. Observe que la forma en que suministramos la entrada ahora es como una `list` de varias `list`. La lista externa itera sobre los dispositivos en el grupo representado por `tff.CLIENTS` y ​​las listas internas iteran sobre elementos en la secuencia local de cada dispositivo.

In [ ]:
get_global_temperature_average([[68.0, 70.0], [71.0], [68.0, 72.0, 70.0]])

70.0

Esto concluye la primera parte del tutorial... le recomendamos que continúe con la [segunda parte](custom_federated_algorithms_2.ipynb).